# Analiza prometnih nesreč v Sloveniji

In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [3]:
from mapboxgl.viz import *
from mapboxgl.utils import *

In [4]:
# Import data
prometne_nesrece1 = pd.read_csv('data/novi_pod/pn2005.csv', sep='$', dtype={"SifraCesteNaselja" : str, 
                                                                            "StacionazaDogodka" : str,
                                                                            "SifraOdsekaUlice" : str})

In [5]:
# Import data
prometne_nesrece2 = pd.read_csv('data/novi_pod/pn06-17.csv', sep='$', dtype={"SifraCesteNaselja" : str, 
                                                                            "StacionazaDogodka" : str,
                                                                            "SifraOdsekaUlice" : str})

In [6]:
prometne_nesrece = pd.concat([prometne_nesrece1, prometne_nesrece2])
prometne_nesrece['DatumPN'] = pd.to_datetime(prometne_nesrece['DatumPN'], format='%d.%m.%Y')
prometne_nesrece=prometne_nesrece.sort_values(by=['DatumPN'])
prometne_nesrece['DanVTednu'] = prometne_nesrece['DatumPN'].dt.weekday_name

In [7]:
prometne_nesrece

,FIOStevilkaZadeve,KlasifikacijaNesrece,UpravnaEnotaStoritve,DatumPN,UraPN,VNaselju,Lokacija,VrstaCesteNaselja,SifraCesteNaselja,TekstCesteNaselja,...,OpisKraja,VzrokNesrece,TipNesrece,VremenskeOkoliscine,StanjePrometa,StanjeVozisca,VrstaVozisca,GeoKoordinataX,GeoKoordinataY,DanVTednu
39752,438244,BREZ POŠKODBE,PTUJ,2005-01-01,13,D,NASELJE,NASELJE Z ULIČNIM SISTEMOM,42135,PTUJ,...,NARAVNO OKOLJE,OSTALO,TRČENJE V OBJEKT,JASNO,NORMALEN,SUHO,ZGLAJEN ASFALT / BETON,141700,565946,Saturday
21962,443991,BREZ POŠKODBE,PIRAN,2005-01-01,10,D,NASELJE,NASELJE BREZ ULIČNEGA SISTEMA,40011,STRUNJAN,...,NARAVNO OKOLJE,PREMIKI Z VOZILOM,TRČENJE V STOJEČE / PARKIRANO VOZILO,JASNO,NEZNANO,SUHO,ZGLAJEN ASFALT / BETON,44079,391239,Saturday
30039,438117,BREZ POŠKODBE,HRASTNIK,2005-01-01,5,D,NASELJE,NASELJE Z ULIČNIM SISTEMOM,10007,HRASTNIK-HRASTNIK,...,NARAVNO OKOLJE,PREMIKI Z VOZILOM,TRČENJE V STOJEČE / PARKIRANO VOZILO,JASNO,NEZNANO,SUHO,HRAPAV ASFALT / BETON,110809,506607,Saturday
32525,442548,BREZ POŠKODBE,LAŠKO,2005-01-01,18,D,NASELJE,NASELJE Z ULIČNIM SISTEMOM,20037,LAŠKO,...,NARAVNO OKOLJE,PREMIKI Z VOZILOM,TRČENJE V STOJEČE / PARKIRANO VOZILO,JASNO,NORMALEN,SUHO,ZGLAJEN ASFALT / BETON,112885,518315,Saturday
1341,438400,BREZ POŠKODBE,SEŽANA,2005-01-01,14,N,CESTA,LOKALNA CESTA,96816,SENOŽEČE-VRABČE-RAŠA,...,CESTA,NEPRILAGOJENA HITROST,ČELNO TRČENJE,JASNO,REDEK,SUHO,ZGLAJEN ASFALT / BETON,67789,422154,Saturday
18281,443361,BREZ POŠKODBE,AJDOVŠČINA,2005-01-01,18,D,NASELJE,NASELJE BREZ ULIČNEGA SISTEMA,01009,DOBRAVLJE-AJDOVŠČINA,...,NARAVNO OKOLJE,OSTALO,TRČENJE V STOJEČE / PARKIRANO VOZILO,NEZNANO,NEZNANO,OSTALO,OSTALO,82794,410822,Saturday
18280,442675,BREZ POŠKODBE,AJDOVŠČINA,2005-01-01,20,D,NASELJE,NASELJE BREZ ULIČNEGA SISTEMA,01009,DOBRAVLJE-AJDOVŠČINA,...,NARAVNO OKOLJE,OSTALO,OSTALO,NEZNANO,NEZNANO,OSTALO,OSTALO,82753,410835,Saturday
6845,438601,BREZ POŠKODBE,LAŠKO,2005-01-01,14,N,CESTA,GLAVNA CESTA I. REDA,00005,PRIKLJ.CELJE ZAHOD-DRNOVO,...,CESTA,OSTALO,TRČENJE V STOJEČE / PARKIRANO VOZILO,JASNO,NORMALEN,SUHO,ZGLAJEN ASFALT / BETON,111416,517944,Saturday
42897,438339,BREZ POŠKODBE,MARIBOR,2005-01-01,10,D,NASELJE,NASELJE Z ULIČNIM SISTEMOM,64033,MARIBOR,...,NARAVNO OKOLJE,NEPRAVILNO PREHITEVANJE,BOČNO TRČENJE,OBLAČNO,REDEK,SUHO,HRAPAV ASFALT / BETON,157182,547604,Saturday
1027,439742,BREZ POŠKODBE,MURSKA SOBOTA,2005-01-01,15,N,CESTA,LOKALNA CESTA,95635,ZENKOVCI-BELI KRIŽ,...,CESTA,PREMIKI Z VOZILOM,TRČENJE V STOJEČE / PARKIRANO VOZILO,JASNO,REDEK,SUHO,ZGLAJEN ASFALT / BETON,180755,582561,Saturday


In [8]:
prometne_nesrece.KlasifikacijaNesrece.unique()

array(['BREZ POŠKODBE', 'LAŽJA TELESNA POŠKODBA', 'HUDA TELESNA POŠKODBA',
       'SMRT'], dtype=object)

In [31]:
dnevi_sum = prometne_nesrece.groupby(['DanVTednu'], as_index = False)
dnevi_sum = dnevi_sum.get_group("Monday").groupby(["KlasifikacijaNesrece"])
dnevi_sum.groups

{'BREZ POŠKODBE': Int64Index([ 27702,  20437,   1291,  52756,  44508,  48922,  25629,  32420,
              19857,  43840,
             ...
             365525, 365438, 365303, 365387, 365431, 365469, 365456, 365416,
             365347, 365937],
            dtype='int64', length=49012),
 'HUDA TELESNA POŠKODBA': Int64Index([ 29866,  49130,   7822,   8764,  28869,  12532,  24251,  20917,
              20705,  42892,
             ...
             363609, 363909, 363900, 364397, 365070, 365104, 365032, 365084,
             365076, 365512],
            dtype='int64', length=1469),
 'LAŽJA TELESNA POŠKODBA': Int64Index([ 32628,   6830,  43481,  42869,  17015,  39182,  33427,  47681,
              26311,  42572,
             ...
             365522, 365196, 365191, 365012, 365150, 365086, 365379, 365419,
             365395, 365920],
            dtype='int64', length=13737),
 'SMRT': Int64Index([ 20820,  15666,   3870,  12078,   5254,  46060,  11512,  11297,
              11481,  10473,
   

In [12]:
dnevi_sum.groups

{'BREZ POŠKODBE': Int64Index([ 39752,  21962,  30039,  32525,   1341,  18281,  18280,   6845,
              42897,   1027,
             ...
             365748, 365809, 365789, 365689, 365803, 365687, 365935, 365934,
             365650, 365832],
            dtype='int64', length=315940),
 'HUDA TELESNA POŠKODBA': Int64Index([ 11977,  15320,  29866,  49130,  52587,   3813,   1930,  27627,
              25098,  28965,
             ...
             365563, 365492, 365332, 365475, 365605, 365745, 365661, 365634,
             365647, 365905],
            dtype='int64', length=11332),
 'LAŽJA TELESNA POŠKODBA': Int64Index([ 37406,   8761,  24210,  51174,   4110,  21402,  18842,  37907,
              28632,   5033,
             ...
             365827, 365628, 365629, 365871, 365901, 365793, 365849, 365627,
             365782, 365691],
            dtype='int64', length=90845),
 'SMRT': Int64Index([  5582,  10727,  35454,  20703,  21808,  28104,  18138,  13185,
              42893,  20526,
 